In [29]:
from __future__ import print_function

import sys
import json
import keras
import numpy as np
from keras.datasets import mnist, cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD

__使用keras中的mnist及cifar10。__

In [31]:
epochs = 1000
num_classes = 0
image_size = 0
image_channel = 0
data_set = ''

__全域變數們，所有訓練最多到1000epochs，並在training step設定early stopping。__

In [41]:
def set_cifar10():
    global image_size, image_channel, num_classes, data_set
    data_set = 'cifar10'
    image_size = 32
    image_channel = 3
    num_classes = 10
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train = scale_pixel_value(np.reshape(crop_image(x_train, crop_size), 
                                       [-1, (image_size - crop_size**2)**2 * image_channel]))
    num_train = x_train.shape[0]
    image_size = image_size - crop_size**2
    return x_train, y_train, x_test, y_test

def set_mnist():
    global image_size, image_channel, num_classes, data_set
    data_set = 'mnist'
    image_size = 28
    image_channel = 1
    num_classes = 10
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(x_train.shape[0], image_size**2 * image_channel)
    x_train = x_train.astype('float32')
    x_train /= 255
    return x_train, y_train, x_test, y_test
    
def crop_image(images, c):
    images = images[:, c:image_size-c, c:image_size-c]
    return images

def scale_pixel_value(images):
    return images/255.

def partially_corrupted_label(label, probability):
    pcl = []
    for l in label:
        if np.random.random_sample() <= probability:
            pcl.append(np.random.randint(0, num_classes))
        else:
            pcl.append(l)
    return np.array(pcl)

def random_label(label):
    return np.reshape(np.random.randint(num_classes, size=len(label)), [-1,])

def label_similarity(label1, label2):
    counter = 0
    print(label1.shape, label2.shape)
    for l1, l2 in zip(label1, label2):
        if l1 != l2:
            counter += 1
    return (len(label1) - float(counter)) / len(label1)

def shuffled_pixel(images):
    shuffled_images = []
    permutation = np.random.permutation(image_size**2 * image_channel)
    for image in images:
        shuffled_image = []
        for p in permutation:
            shuffled_image.append(image[p])
        shuffled_images.append(shuffled_image)
    return np.array(shuffled_images)

def gaussian(images, proportion):
    pixel_num = len(images) * (image_size**2 * image_channel)
    pixel = np.reshape(images, [pixel_num])
    mean = pixel.mean()
    variance_sum = np.ndarray.sum((pixel - mean)**2)
    standard_deviation = np.sqrt(variance_sum / pixel_num)
    temp = np.array(np.random.normal(loc=mean, scale=standard_deviation, 
                                     size=[(int)(len(images)*proportion), (image_size**2 * image_channel)]))
    temp = temp.clip(0.0, 1.0)
    return np.concatenate((images[:(int)(len(images)*(1 - proportion))], temp))
    
def random_pixel(images):
    random_pixel_images = []
    for image in images:
        random_pixel_image = []
        np.random.shuffle(image)
        random_pixel_images.append(image)
    return np.array(random_pixel_images)

In [38]:
def train(x_train, y_train, x_test, y_test, batch_size, hidden_size, layer, learning_rate):

    y_train = keras.utils.to_categorical(y_train, num_classes)
    #y_test = keras.utils.to_categorical(y_test, num_classes)

    model = Sequential()
    model.add(Dense(hidden_size, activation='relu', input_shape=(image_size**2 * image_channel,)))
    
    for i in xrange(layer-1):
        model.add(Dense(hidden_size, activation='relu'))
        #model.add(Dropout(0.2))
    
    model.add(Dense(10, activation='softmax'))
    model.summary()
    
    model.compile(loss='categorical_crossentropy',
                  optimizer=SGD(lr=learning_rate),
                  metrics=['accuracy'])

    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=0)]
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test), 
                        callbacks=callbacks)
    with open('./%s/batch_size_%d_layer_%d_hidden_size_%d_summary' % (data_set, batch_size, layer, hidden_size), 'w') as fout:
        orig_stdout = sys.stdout
        sys.stdout = fout
        print(model.summary())
        sys.stdout = orig_stdout

    return history.history

In [ ]:
x_train , y_train, x_test, y_test = set_mnist()
batch_size = 1024
learning_rate = 0.01
hidden_size = [16, 128, 512, 1024]
layer = [2, 4, 6, 8, 10]

for i in xrange(3):
    for size in hidden_size:
        for l in layer:
            print('hidden_size: %d layer: %d' % (size, l))
            history = train(x_train, y_train, batch_size=batch_size, hidden_size=size, layer=l, learning_rate=learning_rate)
            with open('./%s/%d_batch_size_%d_layer_%d_hidden_size_%d' % (data_set, i+1, batch_size, l, size), 'w') as fout:
                json.dump(history, fout)
    break

for i in xrange(3):
    for size in hidden_size:
        for l in layer:
            print('random label hidden_size: %d layer: %d' % (size, l))
            history = train_mnist(x_train, random_label(y_train), x_test, y_test, batch_size=batch_size, hidden_size=size, layer=l, learning_rate=learning_rate)
            with open('./%s/%d_random_label_batch_size_%d_layer_%d_hidden_size_%d' % (data_set, i+1, batch_size, l, size), 'w') as fout:
                json.dump(history, fout)
    break

gaussian_proportion = [.0, .2, .4, .6, .8, 1.]

for i in xrange(3):
    for proportion in gaussian_proportion:
        for size in hidden_size:
            for l in layer:
                print('gausson: %f hidden_size: %d layer: %d' % (proportion, size, l))
                history = train_mnist(gaussian(x_train, proportion), y_train, x_test, y_test, batch_size=batch_size, hidden_size=size, layer=l, learning_rate=learning_rate)
                with open('./%s/%d_gaussian_%f_batch_size_%d_layer_%d_hidden_size_%d' % (data_set, i+1, proportion, batch_size, l, size), 'w') as fout:
                    json.dump(history, fout)
    break

hidden_size = 512
layer = 3
for i in xrange(3):
    for proportion in gaussian_proportion:
            print('gausson: %f hidden_size: %d layer: %d' % (proportion, size, l))
            history = train_mnist(gaussian(x_train, proportion), random_label(y_train), x_test, y_test, batch_size=batch_size, hidden_size=hidden_size, layer=layer, learning_rate=learning_rate)
            with open('./%s/random_label_gaussian_%f_batch_size_%d_layer_%d_hidden_size_%d' % (data_set, proportion, batch_size, layer, hidden), 'w') as fout:
                json.dump(history, fout)
    break

hidden_size: 16 layer: 2
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_61 (Dense)             (None, 16)                12560     
_________________________________________________________________
dense_62 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_63 (Dense)             (None, 10)                170       
Total params: 13,002
Trainable params: 13,002
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 60000 samples
Epoch 1/1000
60000/60000 [==============================] - 0s - loss: 2.2947 - acc: 0.1190 - val_loss: 2.2628 - val_acc: 0.1697
Epoch 2/1000
60000/60000 [==============================] - 0s - loss: 2.2273 - acc: 0.2131 - val_loss: 2.1837 - val_acc: 0.2615
Epoch 3/1000
60000/60000 [==============================] - 0s - loss

60000/60000 [==============================] - 0s - loss: 0.3415 - acc: 0.9032 - val_loss: 0.3400 - val_acc: 0.9035
Epoch 58/1000
60000/60000 [==============================] - 0s - loss: 0.3391 - acc: 0.9040 - val_loss: 0.3377 - val_acc: 0.9043
Epoch 59/1000
60000/60000 [==============================] - 0s - loss: 0.3367 - acc: 0.9047 - val_loss: 0.3352 - val_acc: 0.9051
Epoch 60/1000
60000/60000 [==============================] - 0s - loss: 0.3345 - acc: 0.9054 - val_loss: 0.3330 - val_acc: 0.9060
Epoch 61/1000
60000/60000 [==============================] - 0s - loss: 0.3323 - acc: 0.9059 - val_loss: 0.3309 - val_acc: 0.9065
Epoch 62/1000
60000/60000 [==============================] - 0s - loss: 0.3302 - acc: 0.9066 - val_loss: 0.3289 - val_acc: 0.9067
Epoch 63/1000
60000/60000 [==============================] - 0s - loss: 0.3281 - acc: 0.9069 - val_loss: 0.3268 - val_acc: 0.9070
Epoch 64/1000
60000/60000 [==============================] - 0s - loss: 0.3261 - acc: 0.9074 - val_loss:

60000/60000 [==============================] - 0s - loss: 0.2656 - acc: 0.9252 - val_loss: 0.2649 - val_acc: 0.9249
Epoch 120/1000
60000/60000 [==============================] - 0s - loss: 0.2649 - acc: 0.9254 - val_loss: 0.2642 - val_acc: 0.9254
Epoch 121/1000
60000/60000 [==============================] - 0s - loss: 0.2642 - acc: 0.9254 - val_loss: 0.2638 - val_acc: 0.9254
Epoch 122/1000
60000/60000 [==============================] - 0s - loss: 0.2636 - acc: 0.9256 - val_loss: 0.2628 - val_acc: 0.9256
Epoch 123/1000
60000/60000 [==============================] - 0s - loss: 0.2629 - acc: 0.9258 - val_loss: 0.2621 - val_acc: 0.9262
Epoch 124/1000
60000/60000 [==============================] - 0s - loss: 0.2621 - acc: 0.9261 - val_loss: 0.2614 - val_acc: 0.9261
Epoch 125/1000
60000/60000 [==============================] - 0s - loss: 0.2615 - acc: 0.9259 - val_loss: 0.2607 - val_acc: 0.9263
Epoch 126/1000
60000/60000 [==============================] - 0s - loss: 0.2608 - acc: 0.9264 - va

60000/60000 [==============================] - 0s - loss: 0.2300 - acc: 0.9356 - val_loss: 0.2296 - val_acc: 0.9359
Epoch 182/1000
60000/60000 [==============================] - 0s - loss: 0.2296 - acc: 0.9356 - val_loss: 0.2291 - val_acc: 0.9359
Epoch 183/1000
60000/60000 [==============================] - 0s - loss: 0.2291 - acc: 0.9356 - val_loss: 0.2288 - val_acc: 0.9360
Epoch 184/1000
60000/60000 [==============================] - 0s - loss: 0.2288 - acc: 0.9363 - val_loss: 0.2282 - val_acc: 0.9363
Epoch 185/1000
60000/60000 [==============================] - 0s - loss: 0.2283 - acc: 0.9361 - val_loss: 0.2278 - val_acc: 0.9364
Epoch 186/1000
60000/60000 [==============================] - 0s - loss: 0.2278 - acc: 0.9364 - val_loss: 0.2274 - val_acc: 0.9363
Epoch 187/1000
60000/60000 [==============================] - 0s - loss: 0.2274 - acc: 0.9365 - val_loss: 0.2269 - val_acc: 0.9364
Epoch 188/1000
60000/60000 [==============================] - 0s - loss: 0.2270 - acc: 0.9364 - va

60000/60000 [==============================] - 0s - loss: 0.2062 - acc: 0.9421 - val_loss: 0.2057 - val_acc: 0.9420
Epoch 244/1000
60000/60000 [==============================] - 0s - loss: 0.2060 - acc: 0.9416 - val_loss: 0.2055 - val_acc: 0.9421
Epoch 245/1000
60000/60000 [==============================] - 0s - loss: 0.2056 - acc: 0.9418 - val_loss: 0.2051 - val_acc: 0.9423
Epoch 246/1000
60000/60000 [==============================] - 0s - loss: 0.2053 - acc: 0.9418 - val_loss: 0.2047 - val_acc: 0.9423
Epoch 247/1000
60000/60000 [==============================] - 0s - loss: 0.2050 - acc: 0.9424 - val_loss: 0.2045 - val_acc: 0.9424
Epoch 248/1000
60000/60000 [==============================] - 0s - loss: 0.2047 - acc: 0.9422 - val_loss: 0.2041 - val_acc: 0.9422
Epoch 249/1000
60000/60000 [==============================] - 0s - loss: 0.2044 - acc: 0.9420 - val_loss: 0.2040 - val_acc: 0.9423
Epoch 250/1000
60000/60000 [==============================] - 0s - loss: 0.2040 - acc: 0.9426 - va

...

60000/60000 [==============================] - 0s - loss: 0.0343 - acc: 0.9917 - val_loss: 0.0341 - val_acc: 0.9918
Epoch 921/1000
60000/60000 [==============================] - 0s - loss: 0.0342 - acc: 0.9919 - val_loss: 0.0340 - val_acc: 0.9920
Epoch 922/1000
60000/60000 [==============================] - 0s - loss: 0.0342 - acc: 0.9919 - val_loss: 0.0340 - val_acc: 0.9919
Epoch 923/1000
60000/60000 [==============================] - 0s - loss: 0.0341 - acc: 0.9920 - val_loss: 0.0339 - val_acc: 0.9920
Epoch 924/1000
60000/60000 [==============================] - 0s - loss: 0.0341 - acc: 0.9919 - val_loss: 0.0339 - val_acc: 0.9920
Epoch 925/1000
60000/60000 [==============================] - 0s - loss: 0.0340 - acc: 0.9919 - val_loss: 0.0339 - val_acc: 0.9921
Epoch 926/1000
60000/60000 [==============================] - 0s - loss: 0.0340 - acc: 0.9918 - val_loss: 0.0338 - val_acc: 0.9921
Epoch 927/1000
60000/60000 [==============================] - 0s - loss: 0.0339 - acc: 0.9920 - va

60000/60000 [==============================] - 0s - loss: 0.0312 - acc: 0.9929 - val_loss: 0.0310 - val_acc: 0.9930
Epoch 983/1000
60000/60000 [==============================] - 0s - loss: 0.0311 - acc: 0.9928 - val_loss: 0.0310 - val_acc: 0.9929
Epoch 984/1000
60000/60000 [==============================] - 0s - loss: 0.0311 - acc: 0.9930 - val_loss: 0.0309 - val_acc: 0.9930
Epoch 985/1000
60000/60000 [==============================] - 0s - loss: 0.0310 - acc: 0.9928 - val_loss: 0.0308 - val_acc: 0.9930
Epoch 986/1000
60000/60000 [==============================] - 0s - loss: 0.0310 - acc: 0.9930 - val_loss: 0.0308 - val_acc: 0.9930
Epoch 987/1000
60000/60000 [==============================] - 0s - loss: 0.0309 - acc: 0.9928 - val_loss: 0.0307 - val_acc: 0.9930
Epoch 988/1000
60000/60000 [==============================] - 0s - loss: 0.0309 - acc: 0.9930 - val_loss: 0.0307 - val_acc: 0.9932
Epoch 989/1000
60000/60000 [==============================] - 0s - loss: 0.0308 - acc: 0.9930 - va

60000/60000 [==============================] - 0s - loss: 0.2759 - acc: 0.9212 - val_loss: 0.2739 - val_acc: 0.9219
Epoch 37/1000
60000/60000 [==============================] - 0s - loss: 0.2723 - acc: 0.9223 - val_loss: 0.2700 - val_acc: 0.9229
Epoch 38/1000
60000/60000 [==============================] - 0s - loss: 0.2687 - acc: 0.9232 - val_loss: 0.2672 - val_acc: 0.9237
Epoch 39/1000
60000/60000 [==============================] - 0s - loss: 0.2654 - acc: 0.9243 - val_loss: 0.2673 - val_acc: 0.9232
Epoch 40/1000
60000/60000 [==============================] - 0s - loss: 0.2620 - acc: 0.9253 - val_loss: 0.2610 - val_acc: 0.9250
Epoch 41/1000
60000/60000 [==============================] - 0s - loss: 0.2587 - acc: 0.9257 - val_loss: 0.2562 - val_acc: 0.9271
Epoch 42/1000
60000/60000 [==============================] - 0s - loss: 0.2559 - acc: 0.9270 - val_loss: 0.2542 - val_acc: 0.9272
Epoch 43/1000
60000/60000 [==============================] - 0s - loss: 0.2529 - acc: 0.9279 - val_loss:

60000/60000 [==============================] - 0s - loss: 0.1529 - acc: 0.9557 - val_loss: 0.1522 - val_acc: 0.9555
Epoch 100/1000
60000/60000 [==============================] - 0s - loss: 0.1519 - acc: 0.9558 - val_loss: 0.1504 - val_acc: 0.9565
Epoch 101/1000
60000/60000 [==============================] - 0s - loss: 0.1507 - acc: 0.9563 - val_loss: 0.1496 - val_acc: 0.9572
Epoch 102/1000
60000/60000 [==============================] - 0s - loss: 0.1496 - acc: 0.9566 - val_loss: 0.1486 - val_acc: 0.9567
Epoch 103/1000
60000/60000 [==============================] - 0s - loss: 0.1486 - acc: 0.9572 - val_loss: 0.1483 - val_acc: 0.9571
Epoch 104/1000
60000/60000 [==============================] - 0s - loss: 0.1477 - acc: 0.9575 - val_loss: 0.1468 - val_acc: 0.9576
Epoch 105/1000
60000/60000 [==============================] - 0s - loss: 0.1466 - acc: 0.9574 - val_loss: 0.1462 - val_acc: 0.9580
Epoch 106/1000
60000/60000 [==============================] - 0s - loss: 0.1458 - acc: 0.9577 - va

60000/60000 [==============================] - 0s - loss: 0.1050 - acc: 0.9701 - val_loss: 0.1052 - val_acc: 0.9698
Epoch 162/1000
60000/60000 [==============================] - 0s - loss: 0.1045 - acc: 0.9703 - val_loss: 0.1036 - val_acc: 0.9707
Epoch 163/1000
60000/60000 [==============================] - 0s - loss: 0.1039 - acc: 0.9703 - val_loss: 0.1030 - val_acc: 0.9705
Epoch 164/1000
60000/60000 [==============================] - 0s - loss: 0.1033 - acc: 0.9707 - val_loss: 0.1028 - val_acc: 0.9708
Epoch 165/1000
60000/60000 [==============================] - 0s - loss: 0.1029 - acc: 0.9703 - val_loss: 0.1022 - val_acc: 0.9710
Epoch 166/1000
60000/60000 [==============================] - 0s - loss: 0.1022 - acc: 0.9705 - val_loss: 0.1017 - val_acc: 0.9710
Epoch 167/1000
60000/60000 [==============================] - 0s - loss: 0.1018 - acc: 0.9712 - val_loss: 0.1015 - val_acc: 0.9710
Epoch 168/1000
60000/60000 [==============================] - 0s - loss: 0.1011 - acc: 0.9712 - va